In [93]:
from pythainlp.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.utils import *
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import os

In [74]:
# Load Datasets
dataframe = pandas.read_csv("Datasets.csv", header=None)
dataset = dataframe.values

In [75]:
# Seperate Dataset into input and output datasets
input_dataset = []
output_dataset = []
for data in dataset:
    input_dataset.append(word_tokenize(data[0], engine='deepcut'))
    output_dataset.append(data[1])
print(input_dataset)
flat_input_list = [item for sublist in input_dataset for item in sublist]
flat_set_of_input_list = list(set(flat_input_list))
flat_set_of_output = list(set(output_dataset))
print("Flat set of Input Datasets",flat_set_of_input_list)
print("Output dataset", flat_set_of_output)

[['ไม่', 'ทราบ', 'ว่า', ' ', 'จะ', 'ติดต่อ', 'อาจารย์', 'ได้', 'ยัง', 'ไง'], ['ไม่', 'ทราบ', 'ว่า', ' ', 'จะ', 'ติดต่อ', 'อ.', 'ได้', 'ยัง', 'ไง'], ['ไม่', 'ทราบ', 'ว่า', ' ', 'จะ', 'ติดต่อ', 'ด๊อกเตอร์', 'ได้', 'ยัง', 'ไง'], ['ไม่', 'ทราบ', 'ว่า', ' ', 'จะ', 'ติดต่อ', 'ดร.', 'ได้', 'ยัง', 'ไง'], ['จะ', 'ติดต่อ', 'อาจารย์', 'ได้', 'ยัง', 'ไง'], ['จะ', 'ติดต่อ', 'อ.', 'ได้', 'ยัง', 'ไง'], ['จะ', 'ติดต่อ', 'ด๊อกเตอร์', 'ได้', 'ยัง', 'ไง'], ['จะ', 'ติดต่อ', 'ดร.', 'ได้', 'ยัง', 'ไง'], ['อาจารย์', 'อยู่', 'ไหม', 'ครับ'], ['อาจารย์', 'อยู่', 'ไหม', 'คะ'], ['อ.', 'อยู่', 'ไหม', 'ครับ'], ['อ.', 'อยู่', 'ไหม', 'คะ'], ['คุณ', 'อยู่', 'ไหม', 'ครับ'], ['คุณ', 'อยู่', 'ไหม', 'คะ'], ['ด๊อกเตอร์', 'อยู่', 'ไหม', 'ครับ'], ['ด๊อกเตอร์', 'อยู่', 'ไหม', 'คะ'], ['ดร.อยู่', 'ไหม', 'ครับ'], ['ดร.', 'อยู่', 'ไหม', 'คะ'], ['อาจารย์', 'อยู่', 'ไหน', 'ครับ'], ['อาจารย์', 'อยู่', 'ไหน', 'คะ'], ['อ.', 'อยู่', 'ไหน', 'ครับ'], ['อ.', 'อยู่', 'ไหน', 'คะ'], ['คุณ', 'อยู่', 'ไหน', 'ครับ'], ['คุณ', 'อยู่', 'ไหน', 'คะ'

In [76]:
# Encode All of input datasets words from string to number
encoder_input = LabelEncoder()
encoder_input.fit(flat_set_of_input_list)
np.save("Encoded_Input_classes.npy" , encoder_input.classes_) # Save Encoded Model
print("Encoder Input Classes :",encoder_input.classes_)

Encoder Input Classes : [' ' 'ครับ' 'คะ' 'คุณ' 'จะ' 'ดร.' 'ดร.อยู่' 'ด๊อกเตอร์' 'ติดต่อ' 'ทราบ'
 'ทาง' 'ที่' 'ยัง' 'ว่า' 'หรอ' 'ห้อง' 'อ.' 'อยู่' 'อาจารย์' 'ไง' 'ได้' 'ไป'
 'ไม่' 'ไหน' 'ไหม']


In [77]:
# Encode All of output datasets words from string to number
encoder_output = LabelEncoder()
encoder_output.fit(flat_set_of_output)
np.save("Encoded_Output_classes.npy" , encoder_output.classes_) # Save Encoded Model

number_of_category = len(encoder_output.classes_)
print("Encoder Output has", number_of_category, "Classes :",encoder_output.classes_)


Encoder Output has 4 Classes : ['ข้อมูล-คน' 'บุคคล' 'สถานที่-คน' 'สถานที่-สถานที่']


In [78]:
# transform input data to encoded
max_word_lenght = 30
encoded_train_x_datasets = sequence.pad_sequences( [list(encoder_input.transform(sentence)) for sentence in input_dataset], maxlen=max_word_lenght )
print("Encoded Train_X dataset has",len(encoded_train_x_datasets), ":\n", encoded_train_x_datasets)

Encoded Train_X dataset has 44 :
 [[ 0  0  0 ..., 20 12 19]
 [ 0  0  0 ..., 20 12 19]
 [ 0  0  0 ..., 20 12 19]
 ..., 
 [ 0  0  0 ..., 17 23 14]
 [ 0  0  0 ..., 21 10 23]
 [ 0  0  0 ..., 10 23 14]]


In [79]:
# transform output data to encoded
encoded_train_y_datasets = np_utils.to_categorical( encoder_output.transform(output_dataset) )
print("Encoded Train_Y dataset :\n", encoded_train_y_datasets)

Encoded Train_Y dataset :
 [[ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]]


In [88]:
# Train Test Split
seed = 7
np.random.seed(seed)
X_train, X_test, y_train, y_test = train_test_split(encoded_train_x_datasets, encoded_train_y_datasets, test_size=0.33, random_state=seed)

In [91]:
# ML Model Structure
model = Sequential()
model.add(Embedding(max_word_lenght, max_word_lenght))
model.add(LSTM(max_word_lenght, dropout=0.2, recurrent_dropout=0.1))
model.add(Dense(max_word_lenght, activation='relu'))
model.add(Dense(max_word_lenght, activation='sigmoid'))
model.add(Dense(max_word_lenght, activation='sigmoid'))
model.add(Dense(number_of_category, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [92]:
model.fit(X_train, y_train, validation_data=(X_test,y_test),
          batch_size=32,
          epochs=300)

Train on 29 samples, validate on 15 samples
Epoch 1/300
29/29 [==============================] - 3s 88ms/step - loss: 1.4282 - acc: 0.1034 - val_loss: 1.4589 - val_acc: 0.0667
Epoch 2/300
29/29 [==============================] - 0s 588us/step - loss: 1.4220 - acc: 0.1034 - val_loss: 1.4521 - val_acc: 0.0667
Epoch 3/300
29/29 [==============================] - 0s 951us/step - loss: 1.4159 - acc: 0.1034 - val_loss: 1.4455 - val_acc: 0.0667
Epoch 4/300
29/29 [==============================] - 0s 605us/step - loss: 1.4100 - acc: 0.1034 - val_loss: 1.4390 - val_acc: 0.0667
Epoch 5/300
29/29 [==============================] - 0s 691us/step - loss: 1.4043 - acc: 0.1034 - val_loss: 1.4329 - val_acc: 0.0667
Epoch 6/300
29/29 [==============================] - 0s 536us/step - loss: 1.3989 - acc: 0.1034 - val_loss: 1.4266 - val_acc: 0.0667
Epoch 7/300
29/29 [==============================] - 0s 570us/step - loss: 1.3935 - acc: 0.1034 - val_loss: 1.4204 - val_acc: 0.0667
Epoch 8/300
29/29 [=======

Epoch 62/300
29/29 [==============================] - 0s 570us/step - loss: 1.1876 - acc: 0.4828 - val_loss: 1.1799 - val_acc: 0.5333
Epoch 63/300
29/29 [==============================] - 0s 622us/step - loss: 1.1848 - acc: 0.4828 - val_loss: 1.1743 - val_acc: 0.5333
Epoch 64/300
29/29 [==============================] - 0s 743us/step - loss: 1.1770 - acc: 0.4828 - val_loss: 1.1673 - val_acc: 0.5333
Epoch 65/300
29/29 [==============================] - 0s 657us/step - loss: 1.1773 - acc: 0.4828 - val_loss: 1.1618 - val_acc: 0.5333
Epoch 66/300
29/29 [==============================] - 0s 743us/step - loss: 1.1680 - acc: 0.4828 - val_loss: 1.1574 - val_acc: 0.5333
Epoch 67/300
29/29 [==============================] - 0s 709us/step - loss: 1.1617 - acc: 0.4828 - val_loss: 1.1536 - val_acc: 0.5333
Epoch 68/300
29/29 [==============================] - 0s 588us/step - loss: 1.1560 - acc: 0.4828 - val_loss: 1.1482 - val_acc: 0.5333
Epoch 69/300
29/29 [==============================] - 0s 691us

Epoch 123/300
29/29 [==============================] - 0s 657us/step - loss: 0.8656 - acc: 0.7241 - val_loss: 0.9435 - val_acc: 0.6000
Epoch 124/300
29/29 [==============================] - 0s 571us/step - loss: 0.8621 - acc: 0.7241 - val_loss: 0.9419 - val_acc: 0.6000
Epoch 125/300
29/29 [==============================] - 0s 622us/step - loss: 0.8590 - acc: 0.7241 - val_loss: 0.9403 - val_acc: 0.6000
Epoch 126/300
29/29 [==============================] - 0s 536us/step - loss: 0.8559 - acc: 0.7241 - val_loss: 0.9389 - val_acc: 0.6000
Epoch 127/300
29/29 [==============================] - 0s 570us/step - loss: 0.8541 - acc: 0.7241 - val_loss: 0.9375 - val_acc: 0.6000
Epoch 128/300
29/29 [==============================] - 0s 985us/step - loss: 0.8506 - acc: 0.7241 - val_loss: 0.9361 - val_acc: 0.6000
Epoch 129/300
29/29 [==============================] - 0s 570us/step - loss: 0.8474 - acc: 0.7241 - val_loss: 0.9348 - val_acc: 0.6000
Epoch 130/300
29/29 [==============================] - 

Epoch 184/300
29/29 [==============================] - 0s 674us/step - loss: 0.7376 - acc: 0.7241 - val_loss: 0.8816 - val_acc: 0.6000
Epoch 185/300
29/29 [==============================] - 0s 519us/step - loss: 0.7355 - acc: 0.7241 - val_loss: 0.8806 - val_acc: 0.6000
Epoch 186/300
29/29 [==============================] - 0s 536us/step - loss: 0.7345 - acc: 0.7241 - val_loss: 0.8797 - val_acc: 0.6000
Epoch 187/300
29/29 [==============================] - 0s 553us/step - loss: 0.7336 - acc: 0.7241 - val_loss: 0.8789 - val_acc: 0.6000
Epoch 188/300
29/29 [==============================] - 0s 588us/step - loss: 0.7322 - acc: 0.7241 - val_loss: 0.8781 - val_acc: 0.6000
Epoch 189/300
29/29 [==============================] - 0s 570us/step - loss: 0.7304 - acc: 0.7241 - val_loss: 0.8774 - val_acc: 0.6000
Epoch 190/300
29/29 [==============================] - 0s 536us/step - loss: 0.7292 - acc: 0.7241 - val_loss: 0.8767 - val_acc: 0.6000
Epoch 191/300
29/29 [==============================] - 

Epoch 245/300
29/29 [==============================] - 0s 588us/step - loss: 0.5792 - acc: 0.7241 - val_loss: 0.7702 - val_acc: 0.6000
Epoch 246/300
29/29 [==============================] - 0s 882us/step - loss: 0.5776 - acc: 0.7241 - val_loss: 0.7656 - val_acc: 0.6000
Epoch 247/300
29/29 [==============================] - 0s 622us/step - loss: 0.5882 - acc: 0.7241 - val_loss: 0.7610 - val_acc: 0.6000
Epoch 248/300
29/29 [==============================] - 0s 519us/step - loss: 0.5655 - acc: 0.7241 - val_loss: 0.7577 - val_acc: 0.6000
Epoch 249/300
29/29 [==============================] - 0s 640us/step - loss: 0.5768 - acc: 0.7241 - val_loss: 0.7554 - val_acc: 0.6000
Epoch 250/300
29/29 [==============================] - 0s 588us/step - loss: 0.5784 - acc: 0.7241 - val_loss: 0.7520 - val_acc: 0.6000
Epoch 251/300
29/29 [==============================] - 0s 605us/step - loss: 0.5697 - acc: 0.7241 - val_loss: 0.7479 - val_acc: 0.6000
Epoch 252/300
29/29 [==============================] - 

In [85]:
predictions = model.predict(X_test)
pre = []
for x in predictions:
    pre.append(x.tolist().index(max(x)))
# round predictions
rounded = [round(x[0]) for x in predictions]
print(predictions,pre)
model.save('model.h5')

[[  1.82999778e-04   6.08839677e-04   2.17977777e-01   5.14173589e-04]
 [  1.85532583e-04   5.67760610e-04   2.14651152e-01   5.27843309e-04]
 [  3.03752627e-03   3.78180772e-01   2.91852956e-03   1.04442443e-04]
 [  1.84520992e-04   5.84270281e-04   2.16017663e-01   5.21865324e-04]
 [  2.57477254e-01   9.68293229e-04   3.79315490e-04   3.59242433e-04]
 [  3.04073957e-03   3.78329217e-01   2.91628088e-03   1.04435872e-04]
 [  3.03796516e-03   3.78220141e-01   2.91824620e-03   1.04441147e-04]
 [  3.03655025e-03   3.78181428e-01   2.91923853e-03   1.04443636e-04]
 [  1.84590681e-04   5.81645756e-04   2.15921119e-01   5.22883027e-04]] [2, 2, 1, 2, 0, 1, 1, 1, 2]
